# Programmieraufgabe 12: Strafverfahren

Als Verfahren für restringierte Optimierungsprobleme wollen wir nun das quadratische Strafverfahren implementieren.

Tragen Sie zunächst in der folgenden Zelle **beide** Ihre Namen ein:

In [ ]:
# Numerische Optimierungsverfahren der Wirtschaftsmathematik
# Wintersemester 2025/2026
# Übungsblatt 12 - Programmieraufgabe 12
#
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]
# 
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]

Führen Sie die folgende Zelle zu Beginn einmal aus, selbst wenn Sie ihre Importe selbst definieren. Sie werden unter anderem auch für das Plotting benötigt.

In [ ]:
import numpy as np
import numpy.linalg as la

import matplotlib.pyplot as plt
import matplotlib.backend_bases

Strafverfahren lösen zur Optimierung von *restringierten* Problemen in jedem Teilschritt ein *unrestringiertes* Optimierungsproblem. Implementieren Sie dafür zunächst ein geeignetes Verfahren.
Beachten Sie dabei:
- Um Verwechslungen mit den Nebenbedingungen zu vermeiden, benutzen wir den Variablennamen `df` statt `g` für den Gradienten.
- Sie können Ihre Implementierung aus einer beliebigen vorherigen Aufgabe wiederverwenden, z.B. das Gradientenverfahren (mit passender Schrittweite), (globalisiertes) BFGS, etc.
- Achten Sie dabei aber vorallem auf den passenden Rückgabewert der Funktion und die nötigen Parameter.
- Entscheiden Sie sich selbst für ein passendes Abbruchkriterium, z.B. $\|f(x^{(k + 1)}) - f(x^{(k)})\| < \text{tol}$. 
- Falls sie z.B. das Newtonverfahren verwenden wollen, können Sie die Signatur der Funktion etc. entsprechend anpassen. Sie müssen dann allerdings auch die Hesse-Matrix für das Strafverfahren und die jeweiligen Testfunktionen und Nebenbedingungen implementieren.

In [ ]:
def global_optimizer(f, df, x_0, tol = 1e-8, kmax = 50):
    '''
        Optimierungsverfahren für das unrestringierte Problem

        Parameter:
            f          : Die zu minimierende Funktion f
            df         : Funktion, die den Gradienten von f zurückgibt
            x_0        : Startvektor
            tol        : Abbruchkriterium
            kmax       : Maximale Zahl an Iterationen

        Rückgabewert:
            x          : Lösung des unrestringierten Problems
    '''
        
    return x

Als nächstes implementieren wir Testfunktionen und deren Nebenbedingungen.

Für die Plotting-Funktionalitäten ist es hier notwendig, dass der Rückgabewert von `f` kein Skalar, sondern ein `np.array` mit einem einzelnen Eintrag und der Dimension `(1,)` ist.

Für die übrigen Funktionen soll folgendes gelten:
- Sei `n` die Dimension des Problems, also $f: \mathbb{R}^n \to \mathbb{R}$ (in dieser Aufgabe gilt immer $n = 2$).
- Weiter nehmen wir an, dass es `m` Ungleichheits- und `p` Gleichheitsnebenbedingungen gibt, also $g: \mathbb{R}^n \to \mathbb{R}^m$ und $h: \mathbb{R}^n \to \mathbb{R}^p$.
- Dann sollen die Rückgabewerte der Funktionen der Form `df_` die Dimension `(n, )` haben, `g_` die Dimension `(m, )`, `dg_` die Dimension `(m, n)`, `h_` die Dimension `(p, )` und `dh_` die Dimension `(p, n)`.
- Falls ein Optimierungsproblem keine Gleichungs- oder Ungleichungsnebenbedinungen $h$ bzw. $g$ hat, verwenden wir `None`.

Sie können das folgende Optimierungsproblem als Vorlage für die beiden anderen verwenden:

$f_1(x, y) = x^2 + y(y+4)$ und 
$g_1(x, y) = -y$


In [ ]:
def f_1(x):
    return np.array([
        x[0]**2 + x[1] * (x[1] + 4.0),
    ])

def df_1(x):
    return np.array([
        2 * x[0], 2 * x[1] + 4.0,
    ])

def g_1(x):
    return np.array([
        -x[1],
    ])

def dg_1(x):
    return np.array([
        [ 0.0, -1.0],
    ])

problem_1 = (f_1, g_1, None, df_1, dg_1, None)

$f_2(x, y) = x(x-10) + y(5y + 4x - 20)$ und 
$h_2(x, y) = x + y - 2$

In [ ]:
def f_2(x):
    return np.array([
        #f(x),
    ])

def df_2(x):
    return np.array([
        #df_dx1(x), #df_dx2(x),
    ])

def h_2(x):
    return np.array([
        #h(x), 
    ])

def dh_2(x):
    return np.array([
        #[dh_dx1(x), dh_dx2(x)],
    ])

problem_2 = (f_2, None, h_2, df_2, None, dh_2)

$f_3(x, y) = x^2 + y(y+4)$ und
$g_3(x, y) = \begin{bmatrix} -y \\ y + 2(x(1-x) + 0.75) \end{bmatrix}$

In [ ]:
def f_3(x):
    return np.array([
        #f(x),
    ])

def df_3(x):
    return np.array([
         #df_dx1(x), #df_dx2(x),
    ])

def g_3(x):
    return np.array([
         #g1(x),
         #g2(x),
    ])

def dg_3(x):
    return np.array([
        #[ dg1_dx1(x),  dg1_dx2(x)],
        #[ dg2_dx1(x),  dg2_dx2(x)],
    ])

problem_3 = (f_3, g_3, None, df_3, dg_3, None)

Für das Strafverfahren benötigen wir eine passende Straffunktion und deren Gradienten. Wir verwenden in dieser Aufgabe die quadratische Straffunktion
$$s(x) = \frac{1}{2} \left(\sum_{i = 1}^m(\max\{0, g_i(x)\})^2 + \sum_{j=1}^p\vert h_j(x)\vert^2 \right) = \frac{1}{2}\left(\Vert g_+(x) \Vert^2 + \Vert h(x) \Vert^2 \right).$$

Implementieren Sie im Folgenden die Straffunktion $s$ als `quadratic_penalty` und ihren Gradienten als `d_quadratic_penalty`. 
- Achten Sie darauf, dass sowohl 'g' als auch 'h' (als auch beide) den Wert `None` haben können. Sie können das z.B. mit
  ```
  if g is not None:
      # Code für ||g_+(x)||^2
  if h is not None:
      # Code für ||h(x)||^2
  ```
  überprüfen.
- Das selbe gilt für die entsprechenden Gradienten `dh` und `dg`. Falls `g` nicht `None` ist, dürfen Sie annehmen, dass auch `dg` nicht `None` ist, und genauso für `h` und `dh`.
- Wie oben definiert hat für ein $n$-dimensionales Optimierungsproblem mit $m$ Ungleichheits- und $p$ Gleichheitsnebenbedingungen `g(x)` die Dimension `(m,)` und `dg(x)` die Dimension `(m, n)`. Genauso gilt `h(x).shape == (p,)` und `dh(x).shape == (p, n)`.

In [ ]:
def quadratic_penalty(g, h, x):
    '''
        Quadratische Straffunktion s
        
        Parameter:
            g       : Funktion, die die Ungleichheitsnebenbedingungen festlegt
            h       : Funktion, die die Gleichheitsnebenbedingungen festlegt
            x       : Funktionswert

        Rückgabewert:
            penalty : Wert von s an der Stelle x
    '''
    
    ???

In [ ]:
def d_quadratic_penalty(g, h, dg, dh, x):
    '''
        Gradient der quadratischen Straffunktion ds
        
        Parameter:
            g         : Ungleichheitsnebenbedingungen
            h         : Gleichheitsnebenbedingungen
            dg        : Gradient von g
            dh        : Gradient von h
            x         : Funktionswert
        Rückgabewert:
            d_penalty : Wert des Gradienten ds an der Stelle x
                        als np.array mit shape (x.shape[0], )
    '''

    ???

Sie können die folgende Zelle als Test verwenden. 

Beachten Sie wie immer, dass dieser Test nur als Hilfestellung gedacht ist. Selbst wenn Ihre Implementierung hier funktioniert, muss sie trotzdem noch nicht immer die korrekte Lösung liefern.

In [ ]:
x = np.array([0.0, -0.5])
assert np.isclose(quadratic_penalty(g_1, None, x), 0.125), "Falscher Wert von s!"
assert np.all(np.isclose(d_quadratic_penalty(g_1, None, dg_1, None, x), [0, -0.5])), "Falscher Wert von ds!"
assert np.isclose(quadratic_penalty(None, h_2, x), 6.25), "Falscher Wert von s!"
assert np.all(np.isclose(d_quadratic_penalty(None, h_2, None, dh_2, x), [-2.5, -2.5])), "Falscher Wert von ds!"
assert np.isclose(quadratic_penalty(g_3, None, x), 0.625), "Falscher Wert von s!"
assert np.all(np.isclose(d_quadratic_penalty(g_3, None, dg_3, None, x), [2.0, 0.5])), "Falscher Wert von ds!"

# Eine inaktive Nebenbedingung
x = np.array([0.0, 0.5])
assert np.isclose(quadratic_penalty(g_1, None, x), 0.0), "Falscher Wert von s!"
assert np.all(np.isclose(d_quadratic_penalty(g_1, None, dg_1, None, x), [0.0, 0.0])), "Falscher Wert von ds!"
assert np.isclose(quadratic_penalty(g_3, None, x), 2.0), "Falscher Wert von s!"
assert np.all(np.isclose(d_quadratic_penalty(g_3, None, dg_3, None, x), [4.0, 2.0])), "Falscher Wert von ds!"

Implementieren Sie jetzt das Strafverfahren (Algorithmus 4.1). Beachten Sie dabei:
- Um die globale Lösung für das Straf-Hilfsproblem zu bestimmen, benutzen Sie Ihre Implementierung von `global_optimizer` von oben.
- Beachten Sie, dass Sie oben nur den Strafterm $s(x)$ implementiert haben, für das ganze Straf-Hilfsproblem $P(x; c) = f(x) + cs(x)$ aber noch die Funktion $f$ und der Strafparameter $c$ fehlen. Sie können dafür z.B. wie folgt die Funktionen verknüpfen:
```
    penalty = lambda y : quadratic_penalty(g, h, y)
    P = lambda y : f(y) + c * penalty(y)
```
- Für den Gradienten von $P(x;c)$, den Sie für `global_optimizer` ebenfalls benötigen, können Sie ähnlich vorgehen.
- Verwenden Sie als Abbruchkriterium $\frac{1}{2}\Vert g_+(x)\Vert^2 + \frac{1}{2}\Vert h(x)\Vert^2 < \text{tol}$.
- Wählen Sie zum Vergrößern des Strafparameters $c_{k+1} = 2 \cdot c_k$ und als Startpunkt der nächsten Iteration die Lösung der vorherigen, also $x_0^{(k+1)} = \hat{x}^{(k)}$.

In [ ]:
def penalty_method(f, g, h, df, dg, dh, x_0, global_optimizer, c_0 = 1.0, tol = 1e-8, kmax = 20):
    '''
        Strafverfahren

        Parameter:
            f                : Die Funktion f
            g                : Ungleichheitsnebenbedingungen
            h                : Gleichheitsnebenbedingungen
            df               : Gradient von f
            dg               : Gradient von g
            dh               : Gradient von h
            x_0              : Startvektor
            global_optimizer : Optimierungsverfahren zum lösen von P(x;c)
            c_0              : Startwert des Strafparameters
            tol              : Abbruchkriterium
            kmax             : Maximale Anzahl an Iterationen

        Rückgabewert:
            iterates         : Vektor der Iterierten. Dabei ist iterates[0,:]
                               der Startwert, iterates[-1,:] der Rückgabewert
                               und len(iterates) - 1 die Anzahl an Iterationen
    '''

    n = x_0.shape[0]
    iterates = np.zeros((kmax + 1, n))
    x = np.copy(x_0)
    c = c_0
    iterates[0,:] = x
    
    ????

    return iterates

Abschließend können wir die drei Testprobleme lösen. Klicken in die Plots legt dabei einen neuen Startwert `x_0` fest.

Falls die Iterationen dabei unregelmäßig erscheinen, kann das an Ihrem globalen Optimierungsalgorithmus liegen. Zum Beispiel kann es sein, dass das Gradientenverfahren mehr als `kmax` Iterationen benötigt, um das globale Minimum zu finden.

Wenn die Iterationen bei `problem_3` zwischen den beiden Seiten der Parabel hin- und herspringen, kann das auch darauf hindeuten, dass Sie im `global_optimizer` eine passende Schrittweite wählen müssen, um das Verfahren zu globalisieren.

In [ ]:
from util.plotting_12 import plot_iterates

# Die nächsten beiden Zeilen werden für die interaktiven Plots benötigt
%pip install -q ipympl
%matplotlib widget

In [ ]:
plot_iterates(penalty_method, *problem_1, np.array([2.0, -2.0]), global_optimizer)

In [ ]:
plot_iterates(penalty_method, *problem_2, np.array([2.0, -2.0]), global_optimizer)

In [ ]:
plot_iterates(penalty_method, *problem_3, np.array([2.0, -2.0]), global_optimizer)